In [ ]:
# Import necessary libraries
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.applications import EfficientNetB0
from keras.models import Model
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Rescaling, Input, Resizing
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.utils import plot_model

print(tf.__version__)

# Check GPU SupportS
print("\nIs GPU Supported?")
print(tf.test.is_built_with_gpu_support())

# Check TensorFlow Detected Devices
print("\nTensorFlow Detected Devices:")
print(tf.config.list_physical_devices())

# Check CUDA availability
print("\nCUDA Available:")
print(tf.test.is_built_with_cuda())
print()

In [ ]:
# Define the directories for train, test, and validation sets
train_dir = 'dataset/train'
test_dir = 'dataset/test'
val_dir = 'dataset/val'

In [ ]:
# Define batch size and image dimensions
batch_size = 128
img_height = None
img_width = None
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
# Load and preprocess the datasets
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  shuffle=True,
  label_mode='binary',
  seed=123,
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  shuffle=True,
  label_mode='binary',
  seed=123,
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  shuffle=True,
  label_mode='binary',
  seed=123,
  batch_size=410)

class_names = train_ds.class_names
label_map = {class_names.index('fire'): 1, class_names.index('nofire'): 0}

# Define a function to remap the labels
def remap_labels(image, label):
    return image, tf.where(label == 0, 1, 0)  # this will swap 0s and 1s

# Use the `map` function to apply the remap_labels function
train_ds = train_ds.map(remap_labels)
val_ds = val_ds.map(remap_labels)
test_ds = test_ds.map(remap_labels)

In [ ]:
# Configure dataset for performance
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Define a function to create model with unified memory
def create_model(base_model):
    # Freeze the convolutional base
    base_model.trainable = True
    fine_tune_at = 1
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    # Add custom layers on top of the pre-trained model
    inputs = Input(shape=(None, None, 3))
    
    x = Rescaling(1./255)(inputs) # Rescale the input RGB values to [0,1] as normalization
    x = Resizing(256, 256, "bilinear", True)(x)
    x = base_model(x, training = True)
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    output = Dense(1, activation='sigmoid')(x)

    # Create the new model
    model = Model(inputs=inputs, outputs=output)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss="binary_crossentropy",
                  metrics=['accuracy'])
    
    return model

In [ ]:
# Model names for saving
model_names = ['EfficientNetB0']
models = [EfficientNetB0]

In [ ]:
# Train models
for model_name, base_model in zip(model_names, models):
    print(f"Training {model_name} model...")
    
    # Load the pre-trained model without the top (fully connected layers)
    base_model = base_model(weights='imagenet', include_top=False, input_shape=(None, None, 3))
    
    # Create model
    with tf.device('/device:GPU:0'):  # Use GPU for model creation
        model = create_model(base_model)
        plot_model(model, to_file=f'{model_name}_architecture.png', show_shapes=True, show_layer_names=True, show_layer_activations=True, show_trainable=True)
        plot_model(base_model, to_file=f'{model_name}_base_architecture.png', show_shapes=True, show_layer_names=True, show_layer_activations=True, show_trainable=True)

    # Print model summary
    model.summary()

    # Define the callbacks
    checkpoint = ModelCheckpoint(f'best_model_{model_name}.h5', monitor='val_loss', save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=7)

    # Fit the model with callbacks and validation data
    with tf.device('/device:GPU:0'):  # Use GPU for model training
        hist = model.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=[checkpoint, early_stopping])

    print(f"Training {model_name} model completed.")

In [ ]:
# Plotting loss
fig, axes = plt.subplots(2, 1, figsize=(10, 10))

# Plot training loss
axes[0].plot(hist.history['loss'], color='teal', label='Training Loss')
axes[0].plot(hist.history['val_loss'], color='orange', label='Validation Loss')
axes[0].set_title(f'{model_name} Loss', fontsize=20)
axes[0].legend(loc="upper right")

# Plot accuracy
axes[1].plot(hist.history['accuracy'], color='blue', label='Training Accuracy')
axes[1].plot(hist.history['val_accuracy'], color='red', label='Validation Accuracy')
axes[1].set_title(f'{model_name} Accuracy', fontsize=20)
axes[1].legend(loc="lower right")

plt.tight_layout()
plt.savefig(f'{model_name}_training_hist.png')

In [28]:
# Evaluate the models
for model_name in model_names:
    # Load the best model
    best_model = load_model(f'best_model_{model_name}.h5')

    # Evaluate the model on the validation data
    val_loss, val_accuracy = best_model.evaluate(test_ds)

    # Print the validation loss and accuracy
    print(f"Testing {model_name} model:")
    print(f"Test Loss ({model_name}):", val_loss)
    print(f"Test Accuracy ({model_name}):", val_accuracy)
    print("-----------------------------")

1/1 [==============================] - 8s 8s/step - loss: 0.2710 - accuracy: 0.9098
Testing EfficientNetB0 model:
Test Loss (EfficientNetB0): 0.2709668278694153
Test Accuracy (EfficientNetB0): 0.9097561240196228
-----------------------------


In [ ]:
# Define the image augmentation layer
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
  tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
  tf.keras.layers.experimental.preprocessing.RandomContrast(0.1),
  tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1, fill_mode='reflect'),
])

# Add the augmentation layer to your datasets
# Note: Data augmentation should be applied to the training and possibly the validation dataset but not the test dataset
def augment_images(image, label):
    image = data_augmentation(image)
    return image, label

train_ds = train_ds.map(augment_images)
val_ds = val_ds.map(augment_images)

In [ ]:
# Configure dataset for performance
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
for model_name, base_model in zip(model_names, models):
    print(f"Fine Tuning {model_name} model...")

    with tf.device('/device:GPU:0'):  # Use GPU for model creation
        model = load_model(f'best_model_{model_name}.h5')

        # Compile the model
        model.compile(optimizer=Adam(learning_rate=5e-5),
                    loss="binary_crossentropy",
                    metrics=['accuracy'])

    # Define the callbacks
    checkpoint = ModelCheckpoint(f'fine_tuned_model_{model_name}.h5', monitor='val_loss', save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=7)

    # Fit the model with callbacks and validation data
    with tf.device('/device:GPU:0'):  # Use GPU for model training
        hist = model.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=[checkpoint, early_stopping])

    print(f"Fine Tuning Trained {model_name} model completed.")

In [ ]:
# Plotting loss
fig, axes = plt.subplots(2, 1, figsize=(10, 10))

# Plot training loss
axes[0].plot(hist.history['loss'], color='teal', label='Training Loss')
axes[0].plot(hist.history['val_loss'], color='orange', label='Validation Loss')
axes[0].set_title(f'{model_name} Tuning Loss', fontsize=20)
axes[0].legend(loc="upper right")

# Plot accuracy
axes[1].plot(hist.history['accuracy'], color='blue', label='Training Accuracy')
axes[1].plot(hist.history['val_accuracy'], color='red', label='Validation Accuracy')
axes[1].set_title(f'{model_name} Tuning Accuracy', fontsize=20)
axes[1].legend(loc="lower right")

plt.tight_layout()
plt.savefig(f'{model_name}_fine_tuning_hist.png')

In [42]:
# Evaluate the models
for model_name in model_names:
    # Load the best model
    best_model = load_model(f'fine_tuned_model_{model_name}.h5')

    # Evaluate the model on the validation data
    val_loss, val_accuracy = best_model.evaluate(test_ds)

    # Print the validation loss and accuracy
    print(f"Testing {model_name} model:")
    print(f"Test Loss ({model_name}):", val_loss)
    print(f"Test Accuracy ({model_name}):", val_accuracy)
    print("-----------------------------")

1/1 [==============================] - 8s 8s/step - loss: 0.2029 - accuracy: 0.9317
Testing EfficientNetB0 model:
Test Loss (EfficientNetB0): 0.2028810828924179
Test Accuracy (EfficientNetB0): 0.9317073225975037
-----------------------------


In [37]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [38]:
dataset_path = './Brazil'  # Adjusted for clarity
test_dataset = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    label_mode='binary'
)

Found 132 files belonging to 2 classes.


In [41]:
# Evaluate the model on the test dataset.
evaluation_loss, evaluation_accuracy = best_model.evaluate(test_dataset)

print(f'Results for Fine Tuned Model: Loss = {evaluation_loss}, Accuracy = {evaluation_accuracy}')

5/5 [==============================] - 4s 419ms/step - loss: 3.6741 - accuracy: 0.2652
Results for Fine Tuned Model: Loss = 3.6740992069244385, Accuracy = 0.2651515007019043


In [ ]:
# Extract images and labels from the generators
test_images, test_labels = next(iter(test_ds))

In [ ]:
# predict the images
pred = best_model.predict(test_images)

In [ ]:
def plot_prediction(images, predictions, true_labels_tensor):
    # Convert TensorFlow tensor of true labels to numpy array
    true_labels = true_labels_tensor.numpy()
    
    # Convert predictions to binary values (0 or 1)
    predictions_binary = (predictions > 0.5).astype(int)
    
    num_rows = 5
    num_cols = 5
    num_images = min(num_rows * num_cols, len(images))  # Ensure we don't exceed images length
    plt.figure(figsize=(2*num_cols, num_rows))
    
    for i in range(num_images):
        plt.subplot(num_rows,num_cols, i+1)
        plt.grid(False)
        plt.xticks([])
        plt.yticks([])
        
        # Normalize and plot image
        img = images[i]
        img = img / 255.0

        plt.imshow(img, aspect='auto')
        
        predicted_label = int(predictions_binary[i].item())
        true_label = int(true_labels[i].item())
        
        if predicted_label == true_label:
            color = 'green'
        else:
            color = 'red'
        
        # Ensure indexing with integer for class names
        plt.xlabel(f"Pred: {class_names[predicted_label]} (True: {class_names[true_label]})", color=color)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_prediction(test_images, pred, test_labels)